# Transfer Learning

The idea of tranfer learning is to grab one high quality network trained for a similar problem and tweak it to work well for the problem at hand. You might be asking, what is a high quality network or model. Well it depends but generally speaking if you have an image recognition problem there is a yearly competition called ImageNet and winner models are published. For other similar domains many models are already hosted in places like Keras or TensorFlow GitHub repos. Keras itself has a few models wrapped into easy to use classes and well documented. Please have a look on [Keras Application](https://keras.io/applications/). Many other amazing models can be found in [TensorFlow repo](https://github.com/tensorflow/models).

In [1]:
import os
import sys
import glob
import argparse

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import numpy as np
import math

# fixed random seed to have consistent results
np.random.seed(123)

train_dir = "data/train"
val_dir = "data/test"
nb_epoch = 5
batch_size = 30
nb_train_samples = 3000
nb_val_samples = 300
nb_classes = 2


img_width, img_height = 299, 299 #fixed size for InceptionV3

Using TensorFlow backend.


The idea here is simple. Load one high quality pre-trained network and remove last layer that is supposed to d ofinal prediction. Replace that layer with a final layer doing the prediction for our two classes and train only the weights for the final layer. As long as you are using a network trained for similar problem like recognising animals or birds, the first layers before prediction are already trained to understand features and representations that would probably apply well to dogs and cats.

In [4]:
# data prep
train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)
test_datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
)

validation_generator = test_datagen.flow_from_directory(
val_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
)

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In the next step, we will grab Inception V3 network with its pre-trained weights simply remove the last layer and add our own last `Dense` layer.

In [5]:
# setup model
base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final fully connected layer

x = base_model.output
x = GlobalAveragePooling2D()(x) # this layer prevents overfitting and generally recommended between conv layers and dense ones
x = Dense(1024, activation='relu')(x) #new FC layer
predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
model = Model(input=base_model.input, output=predictions)

87875584/87910968 [============================>.] - ETA: 0s  

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


If you would like to imagine the depth and complexity of current network, here you go. 
I guess you will appreciate the idea of transfer learning by the end of this exercise.

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [7]:
# Here, we just mark all layers other the the one(s) we added as non trainable. 
# This is actually recommended and it should speed up training.
"""Freeze all layers and compile the model"""
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Let us do the training

history_tl = model.fit_generator(
train_generator,
nb_epoch=nb_epoch,
samples_per_epoch=nb_train_samples,
validation_data=validation_generator,
nb_val_samples=nb_val_samples,
class_weight='auto')

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=100, validation_steps=300, epochs=5, validation_data=<keras.pre..., class_weight="auto")`
  if __name__ == '__main__':


Epoch 1/5
100/100 [==============================] - 225s - loss: 0.2851 - acc: 0.8843 - val_loss: 0.0740 - val_acc: 0.9756

**
Wow, 97.5% accuracy !!
I think this will do the job for our demo.
**

Next we just need to save the current model as it will be used in [next step of the tutorial](https://github.com/ylashin/deep-learning-end-to-end/blob/master/3.CreateServingDockerImage.md).

In [9]:
model.save("dogs-vs-cats-inception.hdf5")